In [1]:
#import necessary libraries
from igraph import *
from tqdm import tqdm
from joblib import Parallel, delayed

In [43]:
g = Graph(directed=True)
g

In [44]:
print(g)

IGRAPH D--- 0 0 --


In [45]:
def get_details(input_type, month):
    
    #hashes list to store all the hashes
    details_dict = dict()
    
    #read files
    if input_type == "inputs":
        filename = "./Data/edges2010/updated/inputs2010_{}.txt".format(str(month+1))
    if input_type == "outputs":
        filename = "./Data/edges2010/updated/outputs2010_{}.txt".format(str(month+1))

    with open(filename) as file:
        for line in file:
            #split by tab char to get the details present
            line = line.replace('\n', '')
            details = line.split("\t")

            #hash present at the second value
            #append in the list
            details_dict[details[1]] = dict()
            
            if input_type == "inputs":
                details_dict[details[1]]["input"] = details[2:]
                
            if input_type == "outputs":
                details_dict[details[1]]["output"] = details[2:]
            
    return details_dict

In [46]:
#as there is data file for each month
month = 12

#input dict to store all the input details
input_dict = dict()

#output dict to store all the output details
output_dict = dict()

In [47]:
import time
start_time = time.time()

#calling function
inp = Parallel(n_jobs=-1)(delayed(get_details)("inputs", i) for i in tqdm(range(month)))
for d in inp:
    input_dict.update(d)

end_time = time.time()
print("Time taken to read the input file data is {}".format(end_time - start_time))

100%|█████████████████████████████████████████| 12/12 [00:00<00:00, 2488.09it/s]


Time taken to read the input file data is 0.6031889915466309


In [48]:
print("Total details in input dict is: {}".format(len(input_dict)))

Total details in input dict is: 117483


In [49]:
import time
start_time = time.time()

#calling function
inp = Parallel(n_jobs=-1)(delayed(get_details)("outputs", i) for i in tqdm(range(month)))
for d in inp:
    output_dict.update(d)

end_time = time.time()
print("Time taken to read the output file data is {}".format(end_time - start_time))

100%|█████████████████████████████████████████| 12/12 [00:00<00:00, 2534.83it/s]


Time taken to read the output file data is 0.6463065147399902


In [50]:
print("Total details in output dict is: {}".format(len(output_dict)))

Total details in output dict is: 117483


In [51]:
#merge input and output data
for key, value in output_dict.items():
    input_details = input_dict[key]
    output_dict[key]["input"] = input_details["input"]

print("Total details in linked dict is: {}".format(len(output_dict)))

Total details in linked dict is: 117483


In [52]:
for key, value in output_dict.items():
    g.add_vertex(key)

In [53]:
for key, value in tqdm(output_dict.items()):
    #print(key)
    hash_of_transc = key
    input_details_of_hash_of_transc = value["input"]
    output_details_of_hash_of_transc = value["output"]
    #print(input_details_of_hash_of_transc)
    #print(output_details_of_hash_of_transc)
    for i in range(0, len(input_details_of_hash_of_transc), 2):
        input_hash = input_details_of_hash_of_transc[i]
        output_index = input_details_of_hash_of_transc[i+1]
        #print(input_hash)
        #print(output_index)
        if input_hash not in output_dict.keys():
            g.add_vertex(input_hash)
            #print("no address")
        else:
            #g.add_vertex(input_hash)
            get_amounts = output_dict[input_hash]["output"]
            #print(get_amounts)
            amount = get_amounts[2*int(output_index)+1]
            
            g.add_edge(input_hash, key)
            g.es["weights"] = amount
        

100%|██████████████████████████████████| 117483/117483 [16:27<00:00, 118.91it/s]


In [55]:
plot(g, "social_network.pdf")

SystemError: <method 'layout_drl' of 'igraph._igraph.GraphBase' objects> returned a result with an error set